In [1]:
import os, sys
sys.path.append("./beaconrunner")

import numpy as np
import pandas as pd

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

from experiments.run import run
import experiments.templates.rl.experiment as rl_experiment

c:\users\shyam\documents\github\beaconrunnerv2\env\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
from stable_baselines3.common.env_checker import check_env

env = rl_experiment.beacon_env
check_env(env)

c:\users\shyam\documents\github\beaconrunnerv2\env\lib\site-packages\stable_baselines3\common\env_checker.py:272: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
c:\users\shyam\documents\github\beaconrunnerv2\env\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


8 proposing block for slot 1
11 proposing block for slot 2
7 proposing block for slot 3
6 proposing block for slot 4
4 proposing block for slot 5
3 proposing block for slot 6
7 proposing block for slot 7
5 proposing block for slot 8
2 proposing block for slot 9
timestep 100 of run 1
2 proposing block for slot 10
4 proposing block for slot 11
